In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
# Rebuild text
from sklearn_bridge import recreate_text_representation
# take text + labels from train ds
from sklearn_bridge import candidate_dict_to_df

In [3]:
from MLutils import report_to_df, train_evaluate

In [4]:
import pickle, time, seaborn, codecs
import pandas as pd
from sklearn.metrics import f1_score, classification_report

In [5]:
with open('candidates_TrainValTestOutgoing.pickle', 'rb') as f:
    candidate_dict = pickle.load(f)

for splt in candidate_dict.keys():
    print "Split = %i : imported %i candidates" %(splt, len(candidate_dict[splt].keys()))

Split = 0 : imported 12987 candidates
Split = 1 : imported 3230 candidates
Split = 2 : imported 8335 candidates
Split = 3 : imported 79400 candidates


# Load train, dev, test datasets

In [6]:
trim_text = True
trim_window = 5
use_lemmas = True

df_train = candidate_dict_to_df(candidate_dict[0],trim_text=trim_text, window=trim_window, lemmas = use_lemmas )
df_val = candidate_dict_to_df(candidate_dict[1],trim_text=trim_text, window=trim_window, lemmas = use_lemmas )
df_test = candidate_dict_to_df(candidate_dict[2],trim_text=trim_text, window=trim_window, lemmas = use_lemmas )

df_unlab = candidate_dict_to_df(candidate_dict[3],trim_text=trim_text, window=trim_window, lemmas = use_lemmas )


In [7]:
def remove_line_breaks(df):
    df['text'] = map(lambda x: x.replace('\n', ' ' ),df.text)
    return df

In [8]:
def save_dataset_and_get_ids(df, path):
    """Splits data into positive and negative examples and returns a list of all the ids seperated"""
    df = remove_line_breaks(df)
    
    if df.label.any(): #means we have labels
        df_pos = df[df.label==1]
        df_neg = df[df.label==-1]

        df_pos.to_csv(path+'.pos',header=False,index= False,encoding='utf-8', columns=[ 'text'],
                   sep= ' ')
        df_neg.to_csv(path+'.neg',header=False,index= False,encoding='utf-8', columns=[ 'text'],
                   sep= ' ')
        
        #keep the order of indices after reshuffling
        ids = list(df_pos.index)
        ids = ids + list(df_neg.index)
        return ids
    else:
        
        df.to_csv(path+'.nolabels',header=False,index= False,encoding='utf-8', columns=[ 'text'],
                   sep= ' ')
        return list(df.index)
    return

In [9]:
train_ids = save_dataset_and_get_ids(df_train, 'cnn/df_train')

val_ids = save_dataset_and_get_ids(df_val, 'cnn/df_val')

test_ids = save_dataset_and_get_ids(df_test, 'cnn/df_test')

unlab_ids = save_dataset_and_get_ids(df_unlab, 'cnn/df_unlab')

In [10]:
# OLD


# save_pos_neg_df(df_train, 'cnn/df_train')
# train_ids = list(df_train.index)
# save_pos_neg_df(df_val, 'cnn/df_val')
# val_ids = list(df_val.index)
# save_pos_neg_df(df_test, 'cnn/df_test')
# test_ids = list(df_test.index)
# save_pos_neg_df(df_unlab, 'cnn/df_unlab')
# unlab_ids = list(df_unlab.index)

# CNN-text-classification

** data_helpers.py **

In [18]:
import numpy as np
import re
import itertools
from collections import Counter

In [19]:
def clean_str(string):
    """
    Tokenization/string cleaning for datasets.
    Original taken from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    return string.strip().lower()



In [20]:
# Probably not needed, can be loaded directly as df / matrix later


def load_data_and_labels(pos_file, neg_file=None):
    """
    Loads polarity data from files, splits the data into words and generates labels.
    Returns split sentences and labels.
    """
    # Load data from files
    positive_examples = list(codecs.open(pos_file, "r", encoding='utf-8' ).readlines())
    positive_examples = [s.strip() for s in positive_examples]
    if neg_file:
        negative_examples = list(codecs.open(neg_file, "r", encoding='utf-8').readlines())
        negative_examples = [s.strip() for s in negative_examples]
        x_text = positive_examples + negative_examples
    else:
        negative_examples = None
        x_text = positive_examples
        
    # Split by words
    x_text = [clean_str(sent) for sent in x_text]
    x_text = [s.split(" ") for s in x_text]
    # Generate labels
    if neg_file:
        positive_labels = [[0, 1] for _ in positive_examples]
        negative_labels = [[1, 0] for _ in negative_examples]
        y = np.concatenate([positive_labels, negative_labels], 0)
        return [x_text, y]
    else:
        return x_text

In [21]:
def pad_sentences(sentences, padding_word="<PAD/>", sequence_length = 64):
    """
    Pads all sentences to the same length. The length is defined by the longest sentence.
    Returns padded sentences.
    """
    
#     sequence_length = max(len(x) for x in sentences)    #commented this out to avoid problems with building in new training data
    padded_sentences = []
    for i in range(len(sentences)):
        sentence = sentences[i]
        num_padding = max(0,sequence_length - len(sentence))
        if num_padding>0:
            new_sentence = sentence + [padding_word] * num_padding
        else: #crop sentence
            new_sentence = sentence[:sequence_length]
        padded_sentences.append(new_sentence)
    return padded_sentences

In [22]:
def build_vocab(sentences):
    """
    Builds a vocabulary mapping from word to index based on the sentences.
    Returns vocabulary mapping and inverse vocabulary mapping.
    """
    # Build vocabulary
    word_counts = Counter(itertools.chain(*sentences))
    # Mapping from index to word
    vocabulary_inv = [x[0] for x in word_counts.most_common()]
    vocabulary_inv = list(sorted(vocabulary_inv))
    # Mapping from word to index
    vocabulary = {x: i for i, x in enumerate(vocabulary_inv)}
    return [vocabulary, vocabulary_inv]

In [23]:
def build_input_data(sentences, labels=None, vocabulary=None):
    """
    Maps sentences and labels to vectors based on a vocabulary.
    """
    x = np.array([[vocabulary[word] for word in sentence] for sentence in sentences])
    y = np.array(labels)
    return [x, y]

In [24]:
# def load_data_old():
#     """
#     Loads and preprocessed data for the dataset.
#     Returns input vectors, labels, vocabulary, and inverse vocabulary.
#     """
#     # Load and preprocess data
#     sentences, labels = load_data_and_labels()
#     sentences_padded = pad_sentences(sentences)
#     vocabulary, vocabulary_inv = build_vocab(sentences_padded)
#     x, y = build_input_data(sentences_padded, labels, vocabulary)
    
#     return [x, y, vocabulary, vocabulary_inv]

In [25]:
def load_data():
    """
    Loads and preprocessed data for the dataset.
    Returns input vectors, labels, vocabulary, and inverse vocabulary.
    """
    # Load and preprocess data
    train_sentences, train_labels = load_data_and_labels(pos_file='cnn/df_train.pos', neg_file='cnn/df_train.neg')
    val_sentences, val_labels = load_data_and_labels(pos_file='cnn/df_val.pos', neg_file='cnn/df_val.neg')
    test_sentences, test_labels = load_data_and_labels(pos_file='cnn/df_test.pos', neg_file='cnn/df_test.neg')
    unlab_sentences = load_data_and_labels(pos_file='cnn/df_unlab.nolabels')
    
    # sentences to build vocab
    all_sentences = train_sentences+val_sentences+test_sentences+unlab_sentences
    
    
    train_sentences_padded = pad_sentences(train_sentences)
    val_sentences_padded = pad_sentences(val_sentences)
    test_sentences_padded = pad_sentences(test_sentences)
    unlab_sentences_padded = pad_sentences(unlab_sentences)
    
    
    #probably need to create all vocab and then split to val, test etc
    vocabulary, vocabulary_inv = build_vocab(pad_sentences(all_sentences))
    
    x_train, y_train = build_input_data(train_sentences_padded, train_labels, vocabulary)    
    x_val, y_val = build_input_data(val_sentences_padded, val_labels, vocabulary)
    x_test, y_test = build_input_data(test_sentences_padded, test_labels, vocabulary)
    x_unlab, _ = build_input_data(unlab_sentences_padded, val_labels, vocabulary) #passing dummy labels here, not using them afterwards
    
    
    return [x_train, y_train, x_val, y_val, x_test, y_test, x_unlab, vocabulary, vocabulary_inv ]


** model.py **

In [28]:
from keras.layers import Input, Dense, Embedding, Conv2D, MaxPool2D

In [29]:

from keras.layers import Reshape, Flatten, Dropout, Concatenate
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from keras.models import Model

In [30]:
print('Loading data')
# x, y, vocabulary, vocabulary_inv = load_data()

# Define max sequence length / or infer from training set (creates problem in new vocab though)
sequence_length = 64 # 56

X_train, y_train, X_val, y_val, X_test, y_test, X_unlab, vocabulary, vocabulary_inv = load_data()
# sequence_length = X_train.shape[1] # 56

vocabulary_size = len(vocabulary_inv) # 18765


Loading data


** Balance training set (either with class weights or undersampling) **

In [20]:
# sample_weights = dict(pd.Series(map(lambda x: x[0],y_train)).value_counts()) # inverse class_count
# sample_weights

In [21]:
# sample_weights = {0: 3302., 1: 9685.}

In [23]:
# from MLutils import balance_candidates

# X_train_us, y_train_us = balance_candidates(X_train, list(map(lambda x: x[0],y_train)))
# y_train_us = np.array(map(lambda x: np.array([1,0] if x==0 else np.array([0,1])),y_train_us))

** train params **

In [31]:
embedding_dim = 100 # original: 256
filter_sizes = [3,4,5]
num_filters = 512
drop = 0.5

epochs = 8
batch_size = 32
sequence_length = X_train.shape[-1]

** Define F1 (+) as metric **

In [25]:
# class TestCallback(Callback):
#     def __init__(self, test_data):
#         self.test_data = test_data

#     def on_epoch_end(self, epoch, logs={}):
#         x, y = self.test_data
#         loss, acc = self.model.evaluate(x, y, verbose=0)
#         print('\nTesting loss: {}, acc: {}\n'.format(loss, acc))

** model building **

In [32]:
# this returns a tensor
print("Creating Model...")
inputs = Input(shape=(sequence_length,), dtype='int32')
embedding = Embedding(input_dim=vocabulary_size, output_dim=embedding_dim, input_length=sequence_length)(inputs)
reshape = Reshape((sequence_length,embedding_dim,1))(embedding)

conv_0 = Conv2D(num_filters, kernel_size=(filter_sizes[0], embedding_dim), padding='valid', kernel_initializer='normal', activation='relu')(reshape)
conv_1 = Conv2D(num_filters, kernel_size=(filter_sizes[1], embedding_dim), padding='valid', kernel_initializer='normal', activation='relu')(reshape)
conv_2 = Conv2D(num_filters, kernel_size=(filter_sizes[2], embedding_dim), padding='valid', kernel_initializer='normal', activation='relu')(reshape)

maxpool_0 = MaxPool2D(pool_size=(sequence_length - filter_sizes[0] + 1, 1), strides=(1,1), padding='valid')(conv_0)
maxpool_1 = MaxPool2D(pool_size=(sequence_length - filter_sizes[1] + 1, 1), strides=(1,1), padding='valid')(conv_1)
maxpool_2 = MaxPool2D(pool_size=(sequence_length - filter_sizes[2] + 1, 1), strides=(1,1), padding='valid')(conv_2)

concatenated_tensor = Concatenate(axis=1)([maxpool_0, maxpool_1, maxpool_2])
flatten = Flatten()(concatenated_tensor)
dropout = Dropout(drop)(flatten)
output = Dense(units=2, activation='softmax')(dropout)

# this creates a model that includes
model = Model(inputs=inputs, outputs=output)

checkpoint = ModelCheckpoint('weights.{epoch:03d}-{val_acc:.4f}.hdf5', monitor='val_acc', verbose=1, save_best_only=True, mode='auto')
adam = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)



Creating Model...


In [33]:
# epochs=20

In [34]:
# Skip for now - try class weights

In [35]:
from imblearn.under_sampling import RandomUnderSampler

rus = RandomUnderSampler()

y_train_bin = list(map(lambda x: x[1],y_train))
X_train_us, y_train_us = rus.fit_sample(X_train,y_train_bin)


y_train_us = np.array(map(lambda x: [0,1] if x==1 else [1,0],y_train_us)) #convert back to logits

len(X_train_us)

6604

In [36]:
X_train_us.shape

(6604, 64)

In [37]:
y_train_us.shape

(6604, 2)

In [32]:
# # with undersampling 

# model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy', f1])
# print("Traning Model...")
# model.fit(X_train_us, y_train_us, batch_size=batch_size, epochs=epochs, verbose=1, callbacks=[checkpoint], 
#          validation_data=(X_val, y_val),
#          )  # starts training


In [33]:
# use val_labels for y_val

# Fix F1 metric

In [38]:
# from https://medium.com/@thongonary/how-to-compute-f1-score-for-each-epoch-in-keras-a1acd17715a2
import numpy as np
from keras.callbacks import Callback
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score, precision_recall_fscore_support

class Metrics(Callback):

    def __init__(self, val_data, logs={}):
        self.val_f1s = []
        self.val_recalls = []
        self.val_precisions = []
        self.val_data = val_data
        
    def on_epoch_end(self, epoch, logs={}):
        
        val_predict = (np.asarray(self.model.predict(self.val_data[0]))).round()
        val_predict = map(lambda x: int(x[1]),val_predict)
        val_targ = self.val_data[1]
        val_targ = map(lambda x: x[1],val_targ)
        
#         _val_f1 = f1_score(val_targ, val_predict,)
#         _val_recall = recall_score(val_targ, val_predict)
#         _val_precision = precision_score(val_targ, val_predict)
        
        _val_precision,_val_recall,_val_f1, support = precision_recall_fscore_support(val_targ, val_predict)
        
        #due to the fact that the positive class is [0,1], we discard the first elemts (correspond to neg class)
        _val_f1 = _val_f1[1]
        _val_precision = _val_precision[1]
        _val_recall = _val_recall[1]
        
        self.val_f1s.append(_val_f1)
        self.val_recalls.append(_val_recall)
        self.val_precisions.append(_val_precision)
        
        print 'val_targ[:10]',val_targ[:10],'\n', 'val_predict[:10]',val_predict[:10],'\n'
        
        print '_val_f1',_val_f1
        print 'support',support
        
        
        print '\n— val_f1: %.2f — val_precision: %.2f — val_recall %.2f\n' %(_val_f1, _val_precision, _val_recall)
#         print _val_f1
#         print _val_precision
#         print _val_recall
        print ''
        return


In [39]:
metrics = Metrics((X_val,y_val))

model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])
print("Traning Model...")
model.fit(X_train_us, y_train_us, batch_size=batch_size, epochs=epochs, verbose=1, callbacks=[metrics], 
         validation_data=(X_val, y_val),
         )  # starts training


Traning Model...
Train on 6604 samples, validate on 3230 samples
Epoch 1/8
6604/6604 [==============================] - 16s 2ms/step - loss: 0.6803 - acc: 0.5747 - val_loss: 0.6431 - val_acc: 0.7368
val_targ[:10] [1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 
val_predict[:10] [0, 1, 1, 0, 1, 0, 0, 0, 0, 0] 

_val_f1 0.424898511502
support [2525  705]

— val_f1: 0.42 — val_precision: 0.41 — val_recall 0.45


Epoch 2/8
6604/6604 [==============================] - 16s 2ms/step - loss: 0.6347 - acc: 0.6938 - val_loss: 0.6214 - val_acc: 0.6988
val_targ[:10] [1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 
val_predict[:10] [1, 1, 1, 0, 1, 1, 1, 0, 0, 1] 

_val_f1 0.510809451986
support [2525  705]

— val_f1: 0.51 — val_precision: 0.40 — val_recall 0.72


Epoch 3/8
6604/6604 [==============================] - 16s 2ms/step - loss: 0.5613 - acc: 0.7583 - val_loss: 0.5496 - val_acc: 0.7508
val_targ[:10] [1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 
val_predict[:10] [1, 1, 1, 0, 1, 1, 1, 0, 1, 1] 

_val_f1 0.557449147883
support [2525  705]


** quick check if keras metric is ok **

## TODO: it is not!

In [36]:
# #evaluation with 0s

# logits = model.predict(X_val)

# y_val_gold = map(lambda x: x[1], y_val)
# y_val_pred_bin = map(lambda x: 1 if x[1]>=0.5 else 0,logits) #if logit of 1 class >=0.5
# class_report = report_to_df(classification_report(y_val_gold, y_val_pred_bin))

# print class_report

# ~~~~

In [40]:
from sklearn.metrics import classification_report
from MLutils import report_to_df

In [41]:
# Helpers to convert to/from Snorkels -1,1 class label lists

def logits_to_neg_labels(logit_array):
    """Input: [[0.93, 0.07],
       [0,1],
       [0,1]]
       Output: [-1,1,1]
    """
    return list(map(lambda x: 1 if x[1]>=0.5 else -1, logit_array))

def logits_to_bin_labels(logit_array):
    """Input: [[1, 0],
           [0,1],
           [0,1]]
           Output: [0,1,1]
    """
    return list(map(lambda x: 1 if x[1]>=0.5 else 0, logit_array))

def get_positive_logit(logit_array):
    """Input: [[1, 0],
           [0,1],
           [0,1]]
           Output: [0,1,1]
    """
    return list(map(lambda x: x[1], logit_array))



def bin_to_neg_labels(label_list):
    return [-1 if x==0 else 1 for x in label_list]

def neg_to_bin_labels(label_list):
    return [0 if x==-1 else 1 for x in label_list]


In [42]:
def classif_report_from_dicts(true_dict, pred_dict):
    ids = true_dict.keys()
    true_list = map(lambda x: true_dict[x],ids)
    pred_list = map(lambda x: pred_dict[x],ids)
    return classification_report(true_list, pred_list)

In [48]:
results_dict = {}

# label_*_binary -> list of -1, 1
# label_*_prob_dict -> {40197: 0.96, 40198: 0.03, ... } (close to 1 -> True/Relevant class )
# NOTE: positive_labels = [[0, 1] for _ in positive_examples]


# pass predictions as dict & binary list
logits = model.predict(X_val)
# y_val_pred = dict(zip(val_ids, list(map(lambda x: x[1], logits)))) #set the probability for the -1 class

y_val_pred_pos_logit = dict(zip(val_ids, get_positive_logit(logits)))
y_val_pred_bin = dict(zip(val_ids, logits_to_bin_labels(logits))) #only used for evaluation
y_val_pred_neg = dict(zip(val_ids, logits_to_neg_labels(logits))) #only used for evaluation


# evaluation on validation set
y_val_gold = dict(df_val.label)
y_test_gold = dict(df_test.label)


# create classification report based on validation set values
class_report = report_to_df(
    classif_report_from_dicts(y_val_gold, y_val_pred_neg)
                         )
print class_report

#print another class_report

# this is correct - because the logits are ordered by + and -
print report_to_df(
    classification_report(
        logits_to_neg_labels(y_val),
        logits_to_neg_labels(logits)
    )
    )



logits = model.predict(X_test)
y_test_pred_pos_logit = dict(zip(test_ids, get_positive_logit(logits)))
y_test_pred_bin = dict(zip(val_ids, logits_to_bin_labels(logits))) #only used for evaluation
y_test_pred_neg = dict(zip(val_ids, logits_to_neg_labels(logits))) #only used for evaluation

logits = model.predict(X_unlab)
y_unlab_pred_pos_logit = dict(zip(unlab_ids, get_positive_logit(logits)))



# To delete
# logits = model.predict(X_test)
# y_test_pred = dict(zip(test_ids, list(map(lambda x: x[1], logits)))) #set the probability for the -1 class
# y_test_pred_bin = dict(zip(test_ids, list(map(lambda x: int(x[1]>=0.5), logits)))) #only used for evaluation


# logits = model.predict(X_unlab)
# y_unlab_pred = dict(zip(unlab_ids, list(map(lambda x: x[1], logits)))) #set the probability for the -1 class
# y_unlab_pred_bin = dict(zip(unlab_ids, list(map(lambda x: int(x[1]>=0.5), logits)))) #only used for evaluation



results_dict['CNN'] = {"label_val_prob+" : y_val_pred_pos_logit,
                       "label_test_prob+" : y_test_pred_pos_logit,
                       "label_unlab_prob+" : y_unlab_pred_pos_logit,
                        "classification_report": class_report,
                        "f1+": class_report.loc['1','f1-score']
                           }

           precision  recall  f1-score  support
Classes                                        
-1              0.91    0.83      0.87     2525
1               0.54    0.72      0.61      705
avg/total       0.83    0.80      0.81     3230
           precision  recall  f1-score  support
Classes                                        
-1              0.91    0.83      0.87     2525
1               0.54    0.72      0.61      705
avg/total       0.83    0.80      0.81     3230


In [52]:
results_dict['CNN'].keys()

['label_val_prob+',
 'label_test_prob+',
 'classification_report',
 'label_unlab_prob+',
 'f1+']

In [70]:
import pickle

In [71]:
with open('ml_predictions/CNN_test.pkl', 'wb') as f:
    pickle.dump(results_dict, f)